In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Project Title: Enhancing Profit Margins for Superstore
   Introduction: In my capacity as a data analyst with expertise in Python, SQL, and data visualization tools like Tableau, I'm embarking on a mission to enhance profit margins for the Superstore dataset in BigQuery. This project is an opportunity to harness data-driven insights to drive profitability. The primary aim is to identify actionable strategies that can substantially boost Superstore's profitability.
Project Outline:
   #### 1. Data Exploration and Understanding:
•	Commencing with a thorough exploration of the Superstore dataset, I'll delve into its structure, features, and data quality.
•	My goal is to uncover historical sales and profit trends to lay a solid foundation for the analysis.
   #### 2. Customer Segmentation:
•	Leveraging RFM analysis, I intend to segment customers based on their purchasing behavior.
•	The objective here is to evaluate the profitability of each customer segment, allowing me to prioritize those with the highest profit potential.
   #### 3. Product Analysis:
•	I will scrutinize product categories and sub-categories to distinguish high-margin and low-margin products.
•	The aim is to pinpoint products that exert the most significant influence on profit and sales.
   #### 4. Pricing Strategy Optimization:
•	My analysis will encompass an examination of the impact of discounts on profit margins and sales.
•	Subsequently, I will formulate recommendations for optimal pricing strategies tailored to different product categories.
   #### 5. Supply Chain and Inventory Management:
•	To optimize profitability, I will assess inventory turnover rates and identify products with slow-moving inventory.
•	Strategies for improved inventory management will be proposed to reduce costs and minimize stockouts.
   #### 6. Geographical Analysis:
•	Geographical insights will be derived by analyzing sales and profit patterns across regions, cities, and states.
•	The objective is to identify regions with untapped growth potential and opportunities for cost reduction.
   #### 7. Data Visualization:
•	Utilizing my proficiency in Tableau, I'll craft compelling visualizations that effectively communicate key findings.
   #### 8. Recommendations and Action Plan:
•	Summarizing critical insights, I'll provide a concrete action plan to enhance profit margins.
•	My recommendations will be actionable and aligned with the Superstore's overarching goals


In [ ]:
!pip install xlrd

In [ ]:
import pandas as pd
import pandasql as ps
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

## Overview of the Dataset: 
   ### The code starts by loading the dataset from an Excel file using the Pandas library and then displays the first 10 rows to provide an overview of the data.

In [ ]:
# Load your dataset from an Excel file into a pandas DataFrame

data = pd.read_excel('/kaggle/input/eu-superstore-data/Superstore.xls', sheet_name='Orders', engine='xlrd')

In [ ]:
# SQL Query 1 - Overview of the Dataset
query1 = """
SELECT *
FROM data
LIMIT 10;
"""
result1 = ps.sqldf(query1, locals())

print("Query 1 - Overview of the Dataset\n")

result1

## Data Quality Checks:
*    ### Query 2 checks for missing values in various columns.
*    ### Query 3 identifies rows with negative values in sales, quantity, discount, or profit.
*    ### Rows with negative values are replaced with their absolute values.

In [ ]:
# SQL Query 2 - Data Quality Check - Missing Values
query2 = """
SELECT
  COUNT(*) AS total_rows,
  SUM(CASE WHEN "Order ID" IS NULL THEN 1 ELSE 0 END) AS missing_order_id,
  SUM(CASE WHEN "Order Date" IS NULL THEN 1 ELSE 0 END) AS missing_order_date,
  SUM(CASE WHEN "Ship Date" IS NULL THEN 1 ELSE 0 END) AS missing_ship_date,
  SUM(CASE WHEN "Ship Mode" IS NULL THEN 1 ELSE 0 END) AS missing_ship_mode,
  SUM(CASE WHEN "Customer ID" IS NULL THEN 1 ELSE 0 END) AS missing_customer_id,
  SUM(CASE WHEN "Customer Name" IS NULL THEN 1 ELSE 0 END) AS missing_customer_name,
  SUM(CASE WHEN "Segment" IS NULL THEN 1 ELSE 0 END) AS missing_segment,
  SUM(CASE WHEN "City" IS NULL THEN 1 ELSE 0 END) AS missing_city,
  SUM(CASE WHEN "State" IS NULL THEN 1 ELSE 0 END) AS missing_state,
  SUM(CASE WHEN "Country" IS NULL THEN 1 ELSE 0 END) AS missing_country,
  SUM(CASE WHEN "Region" IS NULL THEN 1 ELSE 0 END) AS missing_region,
  SUM(CASE WHEN "Product ID" IS NULL THEN 1 ELSE 0 END) AS missing_product_id,
  SUM(CASE WHEN "Category" IS NULL THEN 1 ELSE 0 END) AS missing_category,
  SUM(CASE WHEN "Sub-Category" IS NULL THEN 1 ELSE 0 END) AS missing_sub_category,
  SUM(CASE WHEN "Product Name" IS NULL THEN 1 ELSE 0 END) AS missing_product_name,
  SUM(CASE WHEN "Sales" IS NULL THEN 1 ELSE 0 END) AS missing_sales,
  SUM(CASE WHEN "Quantity" IS NULL THEN 1 ELSE 0 END) AS missing_quantity,
  SUM(CASE WHEN "Discount" IS NULL THEN 1 ELSE 0 END) AS missing_discount,
  SUM(CASE WHEN "Profit" IS NULL THEN 1 ELSE 0 END) AS missing_profit
FROM data;
"""

result2 = ps.sqldf(query2, locals())

print("\nQuery 2 - Data Quality Check - Missing Values\n")

result2


## Data Quality Check - Negative Values

In [ ]:
# SQL Query 3 - Data Quality Check - Negative Values
query3 = """
SELECT Sales, Quantity, Discount, Profit
FROM data
WHERE Sales < 0 OR Quantity < 0 OR Discount < 0 OR Profit < 0;
"""
result3 = ps.sqldf(query3, locals())

print("\nQuery 3 - Data Quality Check - Negative Values\n")

result3

## Replacing Rows with Negative Values

In [ ]:
# Apply .abs() to columns with potential negative values
data['Sales'] = data['Sales'].abs()
data['Quantity'] = data['Quantity'].abs()
data['Discount'] = data['Discount'].abs()
data['Profit'] = data['Profit'].abs()

# Define the path for saving the result after replacing negative values
result_path = '/kaggle/working/result_after_replacing_negatives.csv'

# Save the entire cleaned DataFrame to a CSV file
data.to_csv(result_path, index=False)

df = pd.read_csv('/kaggle/working/result_after_replacing_negatives.csv')

# Query for rows with negative values
query4 = """
SELECT Sales, Quantity, Discount, Profit
FROM df
WHERE Sales < 0 OR Quantity < 0 OR Discount < 0 OR Profit < 0;
"""
result4 = ps.sqldf(query4, locals())

# Print the result
result4

## Summary Statistics: 
   ### Query 5 calculates summary statistics such as the total number of rows, average sales, average profit, maximum sales, maximum profit, minimum sales, and minimum profit.

In [ ]:
# SQL Query 5 - Summary Statistics
query5 = """
SELECT
  COUNT(*) AS total_rows,
  AVG(Sales) AS avg_sales,
  AVG(Profit) AS avg_profit,
  MAX(Sales) AS max_sales,
  MAX(Profit) AS max_profit,
  MIN(Sales) AS min_sales,
  MIN(Profit) AS min_profit
FROM df;
"""
result5 = ps.sqldf(query5, locals())

print("\nQuery 5 - Summary Statistics\n")

result5

## Customer Segmentation (RFM Analysis):

* ### Query 6 identifies the last order date for each customer.
* ### Query 7 calculates the total number of orders and total sales for each customer.

In [ ]:
# SQL Query 6 - Customer Segmentation (RFM Analysis) - last_order_date
query6 = """
SELECT
  "Customer ID",
  "Customer Name",
  MAX("Order Date") AS last_order_date
FROM df
GROUP BY "Customer ID", "Customer Name";
"""
result6 = ps.sqldf(query6, locals())

print("\nQuery 6 - Customer Segmentation (RFM Analysis) - last_order_date\n")

result6

In [ ]:
# Convert 'last_order_date' to datetime if it's not already
result6['last_order_date'] = pd.to_datetime(result6['last_order_date'])

# Create a new column 'last_order_quarter' by extracting the quarter and year
result6['last_order_quarter'] = result6['last_order_date'].dt.to_period('Q')

# Group by last_order_quarter and count customers
quarterly_customer_count = result6.groupby('last_order_quarter').size()

plt.figure(figsize=(10, 6))
quarterly_customer_count.plot(kind='bar', color='skyblue')
plt.xlabel('Last Order Quarter')
plt.ylabel('Number of Customers')
plt.title('Customer Count by Last Order Quarter')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Customer Segmentation (RFM Analysis) - Frequency and Monetary Value

In [ ]:
# SQL Query 7 - Customer Segmentation (RFM Analysis) - Frequency and Monetary Value
query7 = """
SELECT
  "Customer ID",
  "Customer Name",
  COUNT(DISTINCT "Order ID") AS total_orders,
  SUM(Sales) AS total_sales
FROM df
GROUP BY "Customer ID", "Customer Name";
"""
result7 = ps.sqldf(query7, locals())

print("\nQuery 7 - Customer Segmentation (RFM Analysis) - Frequency and Monetary Value\n")

result7

In [ ]:
# Extract data from the result DataFrame
total_orders = result7['total_orders']
total_sales = result7['total_sales']

plt.figure(figsize=(10, 6))
plt.scatter(total_orders, total_sales, color='skyblue', alpha=0.5)
plt.xlabel('Total Orders')
plt.ylabel('Total Sales')
plt.title('Customer Segmentation: Frequency vs. Monetary Value')
plt.grid(True)
plt.tight_layout()
plt.show()

## Product Analysis:

* ### Query 8 analyzes high-margin products by category and sub-category.
* ### Query 9 provides sales by category and sub-category.

In [ ]:
# SQL Query 8 - Product Analysis - High Margin Products
query8 = """
SELECT
  "Category",
  "Sub-Category",
  AVG(Profit) AS avg_profit
FROM df
GROUP BY "Category", "Sub-Category"
ORDER BY avg_profit DESC;
"""
result8 = ps.sqldf(query8, locals())

print("\nQuery 8 - Product Analysis - High Margin Products\n")

result8

In [ ]:
# Extract data from the result DataFrame
categories = result8['Category'] + " - " + result8['Sub-Category']
avg_profit = result8['avg_profit']

plt.figure(figsize=(12, 6))
plt.barh(categories, avg_profit, color='skyblue')
plt.xlabel('Average Profit')
plt.ylabel('Category - Sub-Category')
plt.title('Product Analysis: High Margin Products')
plt.gca().invert_yaxis()  # Invert the y-axis for better readability
plt.tight_layout()
plt.show()

In [ ]:

# Data
categories = result8['Category']
avg_profit = result8['avg_profit']

# Create a bar plot
plt.figure(figsize=(10, 6))
plt.bar(categories, avg_profit, color='skyblue')
plt.xlabel('Product Category')
plt.ylabel('Average Profit')
plt.title('Average Profit by Product Category')
plt.xticks()
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# SQL Query 9 - Sales by Category and Sub-Category
query9 = """
SELECT
  "Category",
  "Sub-Category",
  SUM(Sales) AS total_sales
FROM df
GROUP BY "Category", "Sub-Category"
ORDER BY "Category", total_sales DESC;
"""
result9 = ps.sqldf(query9, locals())

print("\nQuery 9 - Sales by Category and Sub-Category\n")

result9

In [ ]:
# Data
categories = result9['Category'].unique()
subcategories = result9['Sub-Category'].unique()
total_sales = result9['total_sales']

# Create a dictionary to store sales by sub-category for each category
category_sales = {cat: [] for cat in categories}

# Populate the dictionary
for cat in categories:
    for subcat in subcategories:
        subcat_sales = result9[(result9['Category'] == cat) & (result9['Sub-Category'] == subcat)]['total_sales']
        if not subcat_sales.empty:
            category_sales[cat].append(subcat_sales.iloc[0])
        else:
            category_sales[cat].append(0)

# Create a stacked bar chart
plt.figure(figsize=(12, 6))
bottom = None
for cat in categories:
    plt.barh(subcategories, category_sales[cat], label=cat, left=bottom)
    bottom = category_sales[cat] if bottom is None else [bottom[i] + category_sales[cat][i] for i in range(len(bottom))]

# Set labels, title, and legend
plt.xlabel('Total Sales')
plt.ylabel('Sub-Category')
plt.title('Total Sales by Category and Sub-Category (Stacked Bar Chart)')
plt.legend(title='Category')

# Show the plot
plt.tight_layout()
plt.show()

## Time Series Analysis:

   ### Query 10 performs a time series analysis, grouping sales and profit by month and year.

In [ ]:
# SQL Query 10 - Time Series Analysis
query10 = """
SELECT
  strftime('%Y-%m', "Order Date") AS order_month,
  SUM(Sales) AS total_sales,
  SUM(Profit) AS total_profit
FROM df
GROUP BY order_month
ORDER BY order_month;
"""
result10 = ps.sqldf(query10, locals())

print("\nQuery 10 - Time Series Analysis\n")

result10

In [ ]:
# Data
order_month = result10['order_month']
total_sales = result10['total_sales']
total_profit = result10['total_profit']

# Create a figure and axis
plt.figure(figsize=(12, 6))
ax = plt.gca()

# Plot total sales
ax.plot(order_month, total_sales, label='Total Sales', marker='o', color='skyblue')
ax.set_xlabel('Order Month')
ax.set_ylabel('Total Sales')
ax.set_title('Time Series Analysis - Total Sales and Profit Over Time')

# Plot total profit
ax.plot(order_month, total_profit, label='Total Profit', marker='o', color='orange')

# Add legend
ax.legend()

# Rotate x-axis labels for better readability
plt.xticks(rotation=90)

# Show the plot
plt.tight_layout()
plt.show()


## Customer Behavior Analysis:

* ### Query 12 analyzes customer behavior by counting the total orders and identifying the last purchase date for each customer.
* ### Query 13 evaluates customer loyalty based on the total number of orders and total profit for each customer.
* ### Query 14 lists the top 10 customers by total sales.

In [ ]:
# SQL Query 12 - Customer Behavior Analysis
query12 = """
SELECT
  "Customer ID",
  "Customer Name",
  COUNT(DISTINCT "Order ID") AS total_orders,
  MAX("Order Date") AS last_purchase_date
FROM df
GROUP BY "Customer ID","Customer Name"
ORDER BY last_purchase_date DESC;
"""
result12 = ps.sqldf(query12, locals())

print("\nQuery 12 - Customer Behavior Analysis\n")

result12

In [ ]:
# Define the bin edges and labels for the three groups
bin_edges = [0, 6, 11, 16]  
bin_labels = ['Low Orders (0-5)', 'Medium Orders (6-10)', 'High Orders (11-15)']


# Add a new column 'Order Group' to categorize customers
result12['Order Group'] = pd.cut(result12['total_orders'], bins=bin_edges, labels=bin_labels, right=False)

result12.head(3)


In [ ]:
# Count the number of customers in each group
order_group_counts = result12['Order Group'].value_counts()

# Create a bar plot
plt.figure(figsize=(10, 6))
order_group_counts.plot(kind='bar', color='skyblue')
plt.xlabel('Order Group')
plt.ylabel('Number of Customers')
plt.title('Customer Behavior Analysis by Order Group')
plt.xticks(rotation=0)  # Remove rotation of x-axis labels

# Annotate the bars with the counts
for i, count in enumerate(order_group_counts):
    plt.text(i, count + 5, str(count), ha='center', va='bottom', fontsize=12)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# SQL Query 13 - Customer Loyalty Analysis
query13 = """
SELECT
  "Customer ID",
  "Customer Name",
  COUNT(DISTINCT "Order ID") AS total_orders,
  SUM(Profit) AS total_profit
FROM df
GROUP BY "Customer ID", "Customer Name"
ORDER BY total_profit DESC
LIMIT 15;
"""
result13 = ps.sqldf(query13, locals())

print("\nQuery 13 - Customer Loyalty Analysis\n")

result13

In [ ]:
# Data
total_orders = result13['total_orders']
total_profit = result13['total_profit']
customer_names = result13['Customer Name']

# Create a scatter plot
plt.figure(figsize=(12, 8))
plt.scatter(total_orders, total_profit, color='skyblue', alpha=0.5)
plt.xlabel('Total Orders')
plt.ylabel('Total Profit')
plt.title('Customer Loyalty Analysis')

# Add labels for a subset of customer names to avoid overcrowding
show_labels = 5  # Change this value to control how many labels you want to display
for i in range(show_labels):
    plt.annotate(customer_names.iloc[i], (total_orders.iloc[i], total_profit.iloc[i]), fontsize=10)

# Show the plot
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# SQL Query 14 - Top N Customers by Sales
query14 = """
SELECT
  "Customer ID",
  "Customer Name",
  SUM(Sales) AS total_sales
FROM df
GROUP BY "Customer ID", "Customer Name"
ORDER BY total_sales DESC
LIMIT 10;
"""
result14 = ps.sqldf(query14, locals())

print("\nQuery 14 - Top N Customers by Sales\n")

result14

In [ ]:
# Data
customers = result14['Customer Name']
total_sales = result14['total_sales']

# Create a bar plot
plt.figure(figsize=(10, 6))
plt.bar(customers, total_sales, color='skyblue')
plt.xlabel('Customer Name')
plt.ylabel('Total Sales')
plt.title('Top N Customers by Sales')
plt.xticks(rotation=60)
plt.tight_layout()

# Show the plot
plt.show()

## Pricing Strategy Optimization
* ### Query 15 examines the effect of different discount levels on profit margins. It calculates the average profit for each discount percentage and presents the results in ascending order of discount. This analysis can help in making decisions regarding discount strategies and their impact on overall profitability.
* ### Query 16 focuses on understanding the average discount offered to customers in different segments. It calculates the average discount for each customer segment, which could be useful in tailoring marketing and pricing strategies for specific customer groups.

In [ ]:
# SQL Query 15 - Impact of Discounts on Profit Margins
query15 = """
SELECT
  "Discount",
  AVG(Profit) AS avg_profit
FROM df
GROUP BY "Discount"
ORDER BY "Discount";
"""
result15 = ps.sqldf(query15, locals())

print("\nQuery 15 - Impact of Discounts on Profit Margins\n")

result15

In [ ]:
# Data
discounts = result15['Discount']
avg_profit = result15['avg_profit']

# Create a line plot
plt.figure(figsize=(10, 6))
plt.plot(discounts, avg_profit, marker='o', color='skyblue', linestyle='-')
plt.xlabel('Discount')
plt.ylabel('Average Profit')
plt.title('Impact of Discounts on Profit Margins')
plt.grid(True)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# SQL Query 16 - Average Discount by Segment
query16 = """
SELECT
  "Segment",
  AVG(Discount) AS avg_discount
FROM df
GROUP BY "Segment";
"""
result16 = ps.sqldf(query16, locals())

print("\nQuery 16 - Average Discount by Segment\n")

result16

In [ ]:
# Data
segments = result16['Segment']
avg_discounts = result16['avg_discount']

# Create a bar plot
plt.figure(figsize=(10, 6))
plt.bar(segments, avg_discounts, color='skyblue')
plt.xlabel('Segment')
plt.ylabel('Average Discount')
plt.title('Average Discount by Segment')
plt.xticks()
plt.tight_layout()

# Show the plot
plt.show()

## Supply Chain and Inventory Management
* ### Query 17 is related to supply chain and inventory management. It calculates the inventory turnover rate for various products by counting the number of orders and summing the quantity sold for each product. This information is crucial for optimizing inventory levels and ensuring efficient supply chain operations.

In [ ]:
# SQL Query 17 - Inventory Turnover Rate
query17 = """
SELECT
  "Product ID",
  "Product Name",
  "Category",
  "Sub-Category",
  COUNT(DISTINCT "Order ID") AS total_orders,
  SUM(Quantity) AS total_quantity
FROM df
GROUP BY   "Category", "Sub-Category";
"""
result17 = ps.sqldf(query17, locals())

print("\nQuery 17 - Inventory Turnover Rate\n")

result17

In [ ]:
# Data
categories = result17['Category'] + ' - ' + result17['Sub-Category']
total_orders = result17['total_orders']
total_quantity = result17['total_quantity']

# Create a line graph
plt.figure(figsize=(12, 6))
plt.plot(categories, total_orders, marker='o', label='Total Orders', color='skyblue')
plt.plot(categories, total_quantity, marker='o', label='Total Quantity', color='orange')
plt.xlabel('Category - Sub-Category')
plt.ylabel('Count')
plt.title('Total Orders and Total Quantity by Category and Sub-Category')
plt.xticks(rotation=90)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()

## Geographical Analysis
* ### Query 18 provides insights into sales performance across different regions.
* ### Query 19 delves into the profitability of product categories and sub-categories in different regions. 
* ### Query 20 analyzes profitability at the state level. It calculates the total profit for each state and ranks them by descending profitability.

In [ ]:
# SQL Query 18 - Sales by Region
query18 = """
SELECT
  "Region",
  "Country",
  SUM(Sales) AS total_sales
FROM df
GROUP BY "Region", "Country"
ORDER BY total_sales DESC;
"""
result18 = ps.sqldf(query18, locals())

print("\nQuery 18 - Sales by Region\n")

result18

In [ ]:
# Create a DataFrame with region, country, and total sales
df_map = result18.copy()

# Create a bubble map
fig = px.scatter_geo(
    df_map,
    locations="Country",  # Use "Country" as the location field
    locationmode="country names",  # Specify location mode for country names
    color="Region",  # Color bubbles by region
    size="total_sales",  # Size of bubbles represents total sales
    hover_name="Country",  # Show country name on hover
    projection="natural earth",  # Choose a map projection
    title="Sales by Region and Country"
)

# Customize the map layout
fig.update_geos(
    showcoastlines=True, coastlinecolor="Black",
    showland=True, landcolor="lightgray",
    showocean=True, oceancolor="lightblue",
    showcountries=True, countrycolor="Black"
)

# Show the map
fig.show()


In [ ]:
# SQL Query 19 - Profitable Categories and Sub-Categories by Region
query19 = """
SELECT
  "Region",
  "Category",
  "Sub-Category",
  SUM(Profit) AS total_profit
FROM df
GROUP BY "Region", "Category", "Sub-Category"
ORDER BY "Region", total_profit DESC;
"""
result19 = ps.sqldf(query19, locals())

print("\nQuery 19 - Profitable Categories and Sub-Categories by Region\n")

result19

In [ ]:
# Create a DataFrame with region, category, sub-category, and total profit
df_treemap = result19.copy()

# Create a treemap
fig = px.treemap(
    df_treemap,
    path=["Region", "Category", "Sub-Category"],  # Define the hierarchy
    values="total_profit",  # Values to be represented by the treemap tiles
    color="total_profit",  # Color tiles by total profit
    color_continuous_scale="Viridis",  # Choose a color scale
    title="Profitable Categories and Sub-Categories by Region"
)

# Show the treemap
fig.show()

In [ ]:
# SQL Query 20 - Profitability by State
query20 = """
SELECT
  "State",
  SUM(Profit) AS total_profit
FROM df
GROUP BY "State"
ORDER BY total_profit DESC;
"""
result20 = ps.sqldf(query20, locals())

print("\nQuery 20 - Profitability by State\n")



result20